# Load and install libraries

In [1]:
#Load libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline

# Import Kaggle data

In [ ]:
%%bash
pip install dask_xgboost
pip install kaggle
export KAGGLE_USERNAME=ellastar261
export KAGGLE_KEY=92c0b8137c386da6775be167e8e8ca09

kaggle competitions download -c m5-forecasting-accuracy

unzip -n m5-forecasting-accuracy -d m5-forecasting-accuracy
rm -rf sample_data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Archive:  m5-forecasting-accuracy.zip
  inflating: m5-forecasting-accuracy/calendar.csv  
  inflating: m5-forecasting-accuracy/sales_train_evaluation.csv  
  inflating: m5-forecasting-accuracy/sales_train_validation.csv  
  inflating: m5-forecasting-accuracy/sample_submission.csv  
  inflating: m5-forecasting-accuracy/sell_prices.csv  


100%|██████████| 45.8M/45.8M [00:02<00:00, 21.0MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#input paths for base data
trainpath = '/content/drive/MyDrive/Capstone/Models/m5-forecasting-accuracy/sales_train_evaluation.csv'

In [27]:
start_train_day = 1
end_train_day = 1913
prediction_horizon = 28

In [28]:
#Read data into 
sales = pd.read_csv(trainpath)

In [43]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [30]:
df = sales.loc[:,'d_'+str(start_train_day):'d_'+str(end_train_day)]

In [44]:
df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [41]:
ids = list(sales['id'])

In [45]:
results = pd.DataFrame()
for i in range(len(ids)):
  print(i)
  id = ids[i]
  series = np.array(df.iloc[i,:]) 
  fit1 = ExponentialSmoothing(series, seasonal_periods=7, trend='add', seasonal='add').fit(use_boxcox=False) 
  adddf = pd.DataFrame(data={'id':ids[i],'demand':list(fit1.forecast(prediction_horizon)),'day':[i for i in range(end_train_day+1,end_train_day+prediction_horizon+1)]})
  results = pd.concat([results,adddf],axis=0)
results

0


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:1116: FutureWarning: Setting use_boxcox during fit has been deprecated and will be removed after 0.13. It must be set during model initialization.
  FutureWarning,


1


,id,demand,day
0,HOBBIES_1_001_CA_1_evaluation,0.966749,1914
1,HOBBIES_1_001_CA_1_evaluation,1.077845,1915
2,HOBBIES_1_001_CA_1_evaluation,0.932923,1916
3,HOBBIES_1_001_CA_1_evaluation,0.879880,1917
4,HOBBIES_1_001_CA_1_evaluation,0.954979,1918
5,HOBBIES_1_001_CA_1_evaluation,1.231846,1919
6,HOBBIES_1_001_CA_1_evaluation,1.006578,1920
7,HOBBIES_1_001_CA_1_evaluation,0.970514,1921
8,HOBBIES_1_001_CA_1_evaluation,1.081611,1922
9,HOBBIES_1_001_CA_1_evaluation,0.936688,1923


In [ ]:
fcast = results.pivot(index='id',columns='day',values='demand').reset_index()

In [ ]:
#write to CSV
fcast.to_csv('exp_smooth.csv', index=False) 